# Parsing Melbourne Bike Share XML

Ok, now let's move to the last exercise of this week. In this exercise, we are going to have some hands-on experience with XML files. 
I'm sure you are familiar with the content of the XML file we are using in this exercise, as it's our old friend, the restructured Melbourne Bike Share dataset (in XML). We will practise the process of reading and processing XML data.

Before we start walking you through the rest of the session, make sure you have the following data file in the same folder as this IPython Notebook: 
* Melbourne_bike_share.xml

In this session, we will walk you through the process of extracting bike hub station statistical data from *"Melbourne_bike_share.xml"*. Then we will produce the same Pandas DataFrame as what we did before in the two previous actvities.

As usual, let's eyeball to see how the XML file looks like (I've just opened in it in Chrome, but feel free to use any of your favourite editors &#128522;):

![](./img/xml.png)


If you scroll through the opened file, you will find that the same data used in the CSV/JSON examples have been encompassed in XML syntax, using things called **tags**. The figure above shows a snippet of the data.


After eyeballing the file, you should find that 
data values can be stored in two places in an XML file, which are:
* in between two tags, for example, 
    ```html
        <featurename>Harbour Town - Docklands Dve - Docklands</featurename>
    ```
    where the value is *"Harbour Town - Docklands Dve - Docklands"* for the <featurename> tag.
* as an attribute of a tag, for example,
    ```html
        <coordinates human_address="{&quot;address&quot;:&quot;&quot;,&quot;city&quot;:&quot;&quot;
        ,&quot;state&quot;:&quot;&quot;,&quot;zip&quot;:&quot;&quot;}" 
        latitude="-37.814022" longitude="144.939521" needs_recoding="false"/>
    ```
    where the value of latitude is *-37.814022* and longitude is *144.939521*. 

The attributes in XML files store rich information about a specific tag.
Comparing XML with JSON, you will find that the XML tags and attributes hold data in 
a similar way to the JSON keys. 
The advantage of XML is that each tag in XML can hold more than one attribute, and
more values can be stored in one node. See the *"coordinates"* tag above.

Now, how can we extract data stored either in between tags or as an attribute?
In the following sections, we will demonstrate the process of loading and exploring a XML file, extracting
data from the XML file and storing the data in Pandas DataFrame.
* * * 

## 1. Loading the XML Data

In this task, we are to load the Melbourne Bike Share XML data. Please feel free to use whatever methods that you like and enjoy. 

Here, we demonstrate the process using ElementTree.

In [41]:
import xml.etree.ElementTree as etree    
tree = etree.parse("./Melbourne_bike_share.xml")  

Now let's have a look at the `root` element.

In [42]:
root = tree.getroot()     
root.tag

'response'

As expected, the `root` element is the *"response"* element (see the original XML file). You can also check the number of children of the `root` element by:
```python
    len(root)
```
It will give you one (here is the `row`). 

Now, let's have a look at the children of the `root` element.

In [43]:
for child in root:
    print(child) 

<Element 'row' at 0x120ddcb80>


In [44]:
root[0] # retrieving the list element by indexing

<Element 'row' at 0x120ddcb80>

We can see that the list of the `root` element only contains its direct children element. The children elements of each entry in this list are not included. 
If we want to see how many children `root[0]` has, we can do similarly as in the activities:

In [45]:
len(root[0])

50

In [46]:
root[0][0]

<Element 'row' at 0x120f6cd60>

Oh~ &#128580; that was a bit confusing. 
Indeed, if we look back at the XML file, we will realise that `response` list has one element `row`, 
![xmlfolded](./img/xmlfolded.png)
and this `row` list has 50 elements, which are all named `row` as well, although these two `row` are in different levels. 
![xmlunfolded](./img/xmlunfolded.png)


We should know that we always need to either examine the original XML file to discover the structure, or further traverse the element hierarchy by iteratively printing out all the elements and data contained therein.
Programmatically, accessing it through indexing and/or using loops to print out the children of a node are very common methods one would take, for example:

In [47]:
for child in root[0]:
    print(child)

<Element 'row' at 0x120f6cd60>
<Element 'row' at 0x120f6c400>
<Element 'row' at 0x120f6c040>
<Element 'row' at 0x120f4e7c0>
<Element 'row' at 0x120f73180>
<Element 'row' at 0x120f73e50>
<Element 'row' at 0x111429c20>
<Element 'row' at 0x11140a4a0>
<Element 'row' at 0x120f56a90>
<Element 'row' at 0x120f566d0>
<Element 'row' at 0x120f56360>
<Element 'row' at 0x120f56c20>
<Element 'row' at 0x120f56f40>
<Element 'row' at 0x120f601d0>
<Element 'row' at 0x120f60450>
<Element 'row' at 0x120f606d0>
<Element 'row' at 0x120f60950>
<Element 'row' at 0x120f60bd0>
<Element 'row' at 0x120f60e50>
<Element 'row' at 0x120f6f130>
<Element 'row' at 0x120f6f400>
<Element 'row' at 0x120f6f6d0>
<Element 'row' at 0x120f6f950>
<Element 'row' at 0x120f6fbd0>
<Element 'row' at 0x120f6fe50>
<Element 'row' at 0x120f7a130>
<Element 'row' at 0x120f7a3b0>
<Element 'row' at 0x120f7a630>
<Element 'row' at 0x120f7a8b0>
<Element 'row' at 0x120f7ab30>
<Element 'row' at 0x120f7adb0>
<Element 'row' at 0x120f63090>
<Element

In [48]:
for child in root[0][0]:
    print(child)

<Element 'id' at 0x120f6cb30>
<Element 'featurename' at 0x120f6cea0>
<Element 'terminalname' at 0x120f6c9f0>
<Element 'nbbikes' at 0x120f6c950>
<Element 'nbemptydoc' at 0x120f6cd10>
<Element 'uploaddate' at 0x120f6ccc0>
<Element 'coordinates' at 0x120f6ca40>


Above, we retrieve the children of `root[0][0]` and that should correspond to the first record.

Fortunately, the tags of the retrieved child elements correspond to the column heads of the CSV file we worked with before (though, all in lower case).
Thus, all the tags storing the data we want have been found. 
To confirm it, you can inspect the original XML file. 


## 2. Extracting XML Data into Pandas DataFrame
So far we have loaded XML into an element tree and have also found all the tags that contain the data we want. 
In this task, we are going to extract the records in XML and store them into Pandas DataFrame, as we always do :)

In [49]:
len(root[0])

50

Remember `root[0]` correspoind to the outer `row` element. As shown above, it contains 50 children `row` element, each contains the attributes we need to parse. 
To pull out the data, we can access the element properties: `tag`, `attrib` and `text`.

Below we create a data dictionary `dataDict`, with keys equal to the tags name: 'featurename','id','terminalname','nbbikes','nbemptydoc',and 'uploaddate', the value of each will be use to store a list of text values corresponds to those tags. 
It also contains two keys 'lat' and 'lon' each will be use to store a list of the latitude and longtitude information. 

In [50]:
import pandas as pd
# initialise all the fields to be an empty list
dataDict = {}
dataDict['featurename'] = []
dataDict['id'] = []
dataDict['terminalname'] = []
dataDict['nbbikes'] = []
dataDict['nbemptydoc'] = []
dataDict['uploaddate'] = []
dataDict['lat'] = []
dataDict['lon'] = []

Then, we can loop through the outer `row` element, for each child `row`, we again, loop through the child element, and parse the data accordingly. 

For 'featurename','id','terminalname','nbbikes','nbemptydoc',and 'uploaddate', we only need to append the text value to the list accordinly. 

As mentioned in the introduction section, latitudes and longitudes
are stored as attributes in the '*coordinates*' elements. 
Extracting them we need to access specific attributes that corresponds
to latitude and longitude.
Recall that attributes are dictionaries. 
To extract a specific attribute value, you can use the 
square brackets along with the attribute name as the key to obtain
its value.

In [51]:
for child in root[0]: # for each row element
    for gc in child: # for each tag in each row element
        if (gc.tag != 'coordinates'):
            dataDict[gc.tag].append(gc.text)
        else: # if grand child's tag is coordinates, we need to retrive the attribute values
            dataDict['lat'].append(gc.attrib['latitude'])
            dataDict['lon'].append(gc.attrib['longitude'])        

The last step is to store the extracted data into Pandas DataFrame. There are multiple ways of constructing a DataFrame object. Here you are going to generate a DataFrame by passing a Python dictionary to DataFrame's constructor and setting the index to id.

In [52]:
df = pd.DataFrame(dataDict,index = dataDict['id'])

# rename the columns
df.rename(columns = {'featurename':'Featurename','id':'ID','terminalname':'TerminalName',\
                     'nbbikes':'NBBikes','nbemptydoc':'NBEmptydoc','uploaddate':'UploadDate',\
                     'lat':'Latitude','lon':'Longitude'},inplace = True)

df.index.name = 'ID'
del df['ID'] # delete the extra ID column
df.head()

,Featurename,TerminalName,NBBikes,NBEmptydoc,UploadDate,Latitude,Longitude
ID,,,,,,,
2,Harbour Town - Docklands Dve - Docklands,60000,9,14,1453986006,-37.814022,144.939521
4,Federation Square - Flinders St / Swanston St ...,60001,15,7,1453986006,-37.817523,144.967814
5,Plum Garland Reserve - Beaconsfield Pde - Albe...,60002,16,1,1453986006,-37.84782,144.948196
6,State Library - Swanston St / Little Lonsdale ...,60003,9,2,1453986006,-37.810702,144.964417
7,Bourke Street Mall - 205 Bourke St - City,60004,10,1,1453986006,-37.813088,144.967437


Yay! Hero~ Give yourself some &#128079;&#128079;&#128079;